In [1]:
from flask import Flask, render_template, request, redirect, url_for
import mysql.connector
from mysql.connector import Error
from datetime import datetime



app = Flask(__name__)

def get_db_connection():
    conn = mysql.connector.connect(
        host="localhost",
        user="admin",
        password="helloLayka",
        database="test"
    )
    return conn

conn = get_db_connection()
cursor = conn.cursor()

In [2]:
@app.route('/display_documents')
def display_documents():
    try:
        conn = get_db_connection()
        cursor = conn.cursor()
        
        # Fetch documents from the DOCUMENT table, including initials
        cursor.execute("SELECT name, initials, category_id FROM DOCUMENT ORDER BY category_id")
        documents = cursor.fetchall()
        
        # Fetch categories from the CATEGORY table
        cursor.execute("SELECT id, name FROM CATEGORY")
        categories = cursor.fetchall()
        
        # Create a dictionary to store documents by category
        documents_by_category = {}
        for document in documents:
            category_id = document[2]
            document_name_with_initials = f"<b>{document[1]}</b> - {document[0]}"
            if category_id not in documents_by_category:
                documents_by_category[category_id] = []
            documents_by_category[category_id].append(document_name_with_initials)
        
        # Create a dictionary to store category names
        category_names = {category[0]: category[1] for category in categories}
        
        # Calculate the maximum number of documents in any category
        max_docs = max(len(docs) for docs in documents_by_category.values())
        
        # Return the template with the documents, categories, and max_docs
        return render_template('display_documents.html', 
                               documents_by_category=documents_by_category, 
                               category_names=category_names, 
                               max_docs=max_docs)
    
    except Exception as e:
        return f"An error occurred: {str(e)}"
    
    finally:
        cursor.close()
        conn.close()

## Specification page

goto '/specification/add' to add a spec to the table 

goto '/specification/1' to modify the first spec

In [3]:
@app.route('/specification/<int:spec_id>', methods=['GET', 'POST'])
@app.route('/specification/add', methods=['GET', 'POST'])
def manage_specification(spec_id=None):
    conn = get_db_connection()
    cursor = conn.cursor()

    if request.method == 'POST':
        try:
            # Get form data
            title = request.form['title']
            owner_id = request.form['owner']
            document_id = request.form['document']
            review_state_id = request.form['review_state']  # Ensure this is fetched correctly
            spec_parent_list = request.form.getlist('spec_parent_hidden')

            # Handle removal of spec_parent
            remove_spec_id = request.form.get('remove_spec_parent')
            if remove_spec_id:
                spec_parent_list = [spec_id for spec_id in spec_parent_list if spec_id != remove_spec_id]

            # Handle addition of spec_parent
            add_spec_id = request.form.get('add_spec_parent')
            if add_spec_id and add_spec_id.isdigit():
                if add_spec_id not in spec_parent_list:
                    spec_parent_list.append(add_spec_id)

            spec_parent_str = ",".join(spec_parent_list) if spec_parent_list else None
            current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

            if spec_id is None:
                # Insert a new specification
                insert_query = """
                INSERT INTO SPECIFICATION (title, owner_id, document_id, spec_parent, date, state_id)
                VALUES (%s, %s, %s, %s, %s, %s)
                """
                cursor.execute(insert_query, (title, owner_id, document_id, spec_parent_str, current_datetime, review_state_id))
                conn.commit()
                new_spec_id = cursor.lastrowid
                return redirect(url_for('manage_specification', spec_id=new_spec_id))
            else:
                # Update an existing specification
                update_query = """
                UPDATE SPECIFICATION 
                SET title = %s, owner_id = %s, document_id = %s, spec_parent = %s, date = %s, state_id = %s
                WHERE id = %s
                """
                cursor.execute(update_query, (title, owner_id, document_id, spec_parent_str, current_datetime, review_state_id, spec_id))
                conn.commit()
                print(f"Review State Updated: {review_state_id}")  # Debugging
                return redirect(url_for('manage_specification', spec_id=spec_id))

        except Exception as e:
            return f"An error occurred: {str(e)}"

        finally:
            cursor.close()
            conn.close()

    else:
        # Handle GET request, load the page for adding/editing specifications
        try:
            spec_parent_list = []  # Initialize spec_parent_list

            if spec_id is not None:
                # Fetch the specification details
                query = """
                SELECT 
                    s.id,
                    s.title,
                    o.id AS owner_id,
                    o.name AS owner_name,
                    d.id AS document_id, 
                    d.name AS document_name, 
                    s.spec_parent,
                    s.date,
                    rs.id AS review_state_id,
                    rs.state AS review_state,
                    st.SPEC_TYPE_NAME AS spec_type_name,
                    st.TYPE AS spec_type
                FROM 
                    SPECIFICATION s
                LEFT JOIN 
                    OWNER o ON s.owner_id = o.id
                LEFT JOIN 
                    DOCUMENT d ON s.document_id = d.id
                LEFT JOIN 
                    REVIEW_STATE rs ON s.state_id = rs.id
                LEFT JOIN 
                    SPEC_TYPE st ON s.spec_type_id = st.id
                WHERE 
                    s.id = %s
                """
                cursor.execute(query, (spec_id,))
                specification = cursor.fetchone()

                if specification is None:
                    return "Specification not found.", 404

                # Handle spec_parent field: split comma-separated string into a list of IDs
                spec_parent_list = specification[6].split(",") if specification[6] else []

                # Fetch the title of each linked specification in spec_parent
                if spec_parent_list:
                    spec_placeholders = ','.join(['%s'] * len(spec_parent_list))
                    query = """
                    SELECT 
                        s.id,
                        s.title,
                        d.name AS document_name
                    FROM 
                        SPECIFICATION s
                    LEFT JOIN 
                        DOCUMENT d ON s.document_id = d.id
                    WHERE 
                        s.id IN ({})
                    """.format(spec_placeholders)
                    cursor.execute(query, spec_parent_list)
                    spec_references = cursor.fetchall()

                    # Sort by document name
                    spec_references.sort(key=lambda x: x[2])  # x[2] corresponds to document_name
                else:
                    spec_references = []

                # Fetch child specifications where this spec is a parent
                child_query = """
                SELECT id, title 
                FROM SPECIFICATION
                WHERE spec_parent LIKE %s
                """
                cursor.execute(child_query, (f'%{spec_id}%',))
                child_specs = cursor.fetchall()
            else:
                specification = None
                spec_references = []
                child_specs = []

            # Fetch possible options for dropdowns
            cursor.execute("SELECT id, name FROM OWNER")
            owners = cursor.fetchall()
            cursor.execute("SELECT id, name FROM DOCUMENT")
            documents = cursor.fetchall()
            cursor.execute("SELECT id, state FROM REVIEW_STATE")
            review_states = cursor.fetchall()
            cursor.execute("SELECT id, title FROM SPECIFICATION")
            all_specs = cursor.fetchall()

            return render_template(
                'edit_specification.html',
                specification=specification,
                spec_references=spec_references,  # Linked specifications with titles
                spec_parent_list=spec_parent_list,  # IDs of linked specifications
                owners=owners,
                documents=documents,
                review_states=review_states,
                all_specs=all_specs,
                child_specs=child_specs
            )

        except Exception as e:
            return f"An error occurred: {str(e)}"

        finally:
            cursor.close()
            conn.close()


## Setting page and helpers

In [4]:
@app.route('/setting')
def setting():
    conn = get_db_connection()
    cursor = conn.cursor(dictionary=True)

    # Fetching all rows for OWNER, CATEGORY, DOC_STATE, REVIEW_STATE
    cursor.execute('SELECT * FROM OWNER')
    owner_rows = cursor.fetchall()

    cursor.execute('SELECT * FROM CATEGORY')
    category_rows = cursor.fetchall()

    cursor.execute('SELECT * FROM DOC_STATE')
    doc_state_rows = cursor.fetchall()

    cursor.execute('SELECT * FROM REVIEW_STATE')
    review_state_rows = cursor.fetchall()

    # Query to join DOCUMENT with OWNER and CATEGORY tables
    cursor.execute('''
        SELECT DOCUMENT.id, DOCUMENT.name, DOCUMENT.initials, OWNER.name AS owner_name, CATEGORY.name AS category_name, DOCUMENT.owner_id, DOCUMENT.category_id
        FROM DOCUMENT
        JOIN OWNER ON DOCUMENT.owner_id = OWNER.id
        JOIN CATEGORY ON DOCUMENT.category_id = CATEGORY.id
    ''')
    document_rows = cursor.fetchall()

    # Fetching lists for dropdowns in the DOCUMENT table
    cursor.execute('SELECT id, name FROM OWNER')
    owner_list = cursor.fetchall()

    cursor.execute('SELECT id, name FROM CATEGORY')
    category_list = cursor.fetchall()

    cursor.close()
    conn.close()

    return render_template('setting.html',
                           owner_rows=owner_rows, 
                           category_rows=category_rows, 
                           doc_state_rows=doc_state_rows, 
                           review_state_rows=review_state_rows,
                           document_rows=document_rows,
                           owner_list=owner_list,
                           category_list=category_list)

@app.route('/modify_setting/<table_name>/<int:row_id>', methods=['POST'])
def modify_setting(table_name, row_id):
    conn = get_db_connection()
    cursor = conn.cursor()

    # Build the update query dynamically based on the form fields
    update_fields = ', '.join([f"{key} = %s" for key in request.form.keys()])
    values = list(request.form.values())

    cursor.execute(f"UPDATE {table_name} SET {update_fields} WHERE id = %s", values + [row_id])
    conn.commit()
    cursor.close()
    conn.close()

    return redirect(url_for('setting'))



@app.route('/delete_setting/<table_name>/<int:row_id>', methods=['POST'])
def delete_setting(table_name, row_id):
    conn = get_db_connection()
    cursor = conn.cursor()
    
    try:
        cursor.execute(f'DELETE FROM {table_name} WHERE id = %s', (row_id,))
        conn.commit()
    except Error as e:
        # Check if the error is due to a foreign key constraint
        if e.errno == 1451:  # MySQL error code for foreign key constraint failure
            cursor.close()
            conn.close()
            return f"Cannot delete this item because it is still in use. The {table_name} entry with ID {row_id} could not be deleted because it is referenced by another table.", 400
        else:
            raise e  # Re-raise if it's another kind of error
    
    cursor.close()
    conn.close()
    
    return redirect(url_for('setting'))

@app.route('/add_setting/<table_name>', methods=['POST'])
def add_setting(table_name):
    conn = get_db_connection()
    cursor = conn.cursor()

    # Extract the form data, excluding the id field since it's usually auto-incremented
    keys = ', '.join(request.form.keys())
    placeholders = ', '.join(['%s'] * len(request.form))
    values = list(request.form.values())

    cursor.execute(f"INSERT INTO {table_name} ({keys}) VALUES ({placeholders})", values)
    conn.commit()
    cursor.close()
    conn.close()

    return redirect(url_for('setting'))

In [ ]:
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5001)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://192.168.1.30:5001
Press CTRL+C to quit
192.168.1.18 - - [27/Oct/2024 19:02:13] "GET /specification/add HTTP/1.1" 200 -
192.168.1.18 - - [27/Oct/2024 19:02:17] "GET /display_documents HTTP/1.1" 200 -
192.168.1.18 - - [27/Oct/2024 19:02:21] "GET /specification/add HTTP/1.1" 200 -
192.168.1.18 - - [27/Oct/2024 19:02:23] "GET /setting HTTP/1.1" 200 -
192.168.1.18 - - [27/Oct/2024 19:02:25] "GET /specification/add HTTP/1.1" 200 -
192.168.1.18 - - [27/Oct/2024 19:02:26] "GET /display_documents HTTP/1.1" 200 -
192.168.1.18 - - [27/Oct/2024 19:02:31] "GET /setting HTTP/1.1" 200 -
192.168.1.18 - - [27/Oct/2024 19:02:36] "GET /display_documents HTTP/1.1" 200 -
192.168.1.18 - - [27/Oct/2024 19:02:47] "GET /setting HTTP/1.1" 200 -
192.168.1.18 - - [27/Oct/2024 19:02:56] "POST /modify_setting/CATEGORY/5 HTTP/1.1" 302 -
192.168.1.18 - - [27/Oct/2024 19:02:56] "GET /setting HTTP/1.1" 200 -
192.168.1.18 - - 